In [ ]:
%pylab --no-import-all inline

# Load and preprocess 1976 data

Time to start looking at other years!

---

In [1]:
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Load the data.

---

If this fails, be sure that you've saved your own data in the prescribed location, then retry.

In [2]:
file = "../data/interim/1976data.dta"  
# Matt Wilson converted the older Stata file to the one we use.
df_rawest = pd.read_stata(file)

In [3]:
good_columns = [
    'V763168',  # DO YOU USUALLY THINK OF YOURSELF AS A REPUBLICAN, A DEMOCRAT, AN INDEPENDENT, OR WHAT?
    'V763357',  # 1: keep military spending; 5: raise military spending
    'V763353',  # Gov't should spend less, even if cutting health and education.
    'V763273',  # Private vs public insurance
    'V763911',  # FAVOR STRICT GUN CNTRL (1: favor)
    'V763241',  # GOVT GUAR JOB/S.L  (1: guarantee)
    'V763264',  # MNRTY GRP AID SCL (1: help)
    'V763511',  # HOW OFTEN R ATND CHURCH (1: highest)
    'V763796',  # OPIN:WHEN ALLOW ABORTION (1: never)
    'V763384',  # Educational attainment (0: lowest, 17: highest)
    'V763757',  # THE POOR ARE POOR BECAUSE THE AMERICAN WAY OF LIFE DOESN'T GIVE ALL PEOPLE AN EQUAL CHANCE? (1: agree)
    'V763665',  # WHO DID YOU VOTE FOR IN THE ELECTION FOR PRESIDENT?
    'V763174',  # SUMMARY-R'S PARTY ID
]
df_raw = df_rawest[good_columns]

In [4]:
def convert_to_int(s):
    """Turn ANES data entry into an integer.
    
    >>> convert_to_int("1. Govt should provide many fewer services")
    1
    >>> convert_to_int("2")
    2
    """
    try:
        return int(s.partition('.')[0])
    except ValueError:
        warnings.warn("Couldn't convert: "+s)
        return np.nan
    except AttributeError:
        return s
    
def not_informative_to_nan(x):
    """Convert non-informative values to missing.
    
    ANES codes various non-answers as 8, 9, and 0.
    For instance, if a question does not pertain to the 
    respondent.
    """
    return np.nan if x in {8, 9, 0} else x


def fix_V763168(x):
    return x if x in {1, 2} else 3 if x == 5 else np.nan

df = df_raw.applymap(convert_to_int)
df.drop(['V763384', 'V763174'], axis='columns').applymap(not_informative_to_nan)  # Dropped because its info is different.
df.V763384.apply(lambda x: np.nan if x > 20 else x)
df.V763174.apply(lambda x: x if x in range(7) else np.nan)

df.V763665.apply(lambda x: np.nan if x not in {1, 2} else x);

In [5]:
df.tail()

,V763168,V763357,V763353,V763273,V763911,V763241,V763264,V763511,V763796,V763384,V763757,V763665,V763174
2243,1,5,5,0,0,0,3,2,0,12,0,0,5
2244,1,5,5,3,0,7,5,3,0,12,0,0,5
2245,1,5,5,2,1,7,6,1,1,12,4,1,5
2246,5,5,5,1,5,4,7,3,4,11,5,2,0
2247,3,5,5,7,1,7,6,3,4,12,5,1,3


In [6]:
print("Variables now available: df")

Variables now available: df


In [7]:
df.to_csv("../data/processed/1976.csv")